# 指令问答
让模型给出具体回答的最佳方法之一是改善提示的格式。如前所述，一个提示可以结合指令、上下文、输入和输出指示（output indicators）来获得更好的结果。虽然这些组成部分不是必需的，但随着你给出的指令越具体，你将获得越好的结果，这成为了一种良好的实践。下面是一个例子，展示了按照更结构化的提示词来会是什么样子的。

In [1]:
### 加载模型Qwen模型
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name_or_path_instruct = '/home/lixinyu/weights/Qwen2.5-3B-Instruct'  # 替换为你下载的模型路径
tokenizer_instruct = AutoTokenizer.from_pretrained(model_name_or_path_instruct)
model_instruct = AutoModelForCausalLM.from_pretrained(model_name_or_path_instruct,device_map='cuda:0', torch_dtype='auto')

/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.18it/s]


In [2]:
# 推理函数
def inference(model,tokenizer,prompt,system="你是一个专业的人工智能助手",max_new_tokens=512,temperature=0.9):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
    )
    # 提取仅由模型生成的token ids（排除输入部分）
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    # 统计生成的tokens数量（直接取生成序列的长度）
    generated_tokens_count = len(generated_ids[0])  # 因输入是单条，取第一个元素的长度
    # 解码生成的token ids为文本
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # 返回响应文本和生成的tokens数量
    return {
        "response":response,
        "generated_tokens_count":generated_tokens_count
    }

In [3]:
prompt="""根据下面的上下文回答问题。回答要简明扼要。如果不确定答案，回答“不确定答案”。

Question: 为什么中国古代的 “丝绸之路” 被称为东西方文明交流的重要纽带？
Answer:
"""

In [4]:
print(inference(model_instruct,tokenizer_instruct,prompt))


{'response': '因为“丝绸之路”促进了中国与中亚、西亚乃至欧洲之间的政治、经济和文化交往，是古代东西方文明交流的重要通道。', 'generated_tokens_count': 31}


可以看到其实上面的例子里，提示词的组件并没有很全，那如果我们举一个稍微复杂点的例子，尽量补充提示词组件，比如：

In [7]:
prompt="""角色：你是一位科普作者。  
背景：读者是 15 岁的高中生，没有编程经验。  
任务：用 80 字以内、带一个比喻的方式解释“梯度下降”在神经网络训练中的作用，并给出生活化的例子。  
输出格式：  
1. 比喻句  
2. 生活例子"""

In [8]:
print(inference(model_instruct,tokenizer_instruct,prompt))

{'response': '比喻句：梯度下降就像是一只迷路的小狗，它需要找到回家的路（最小化损失函数），通过不断尝试不同的路径（调整参数），直到找到最佳路线（最小化误差）。\n\n生活例子：想象你在找一片美味的宝藏（最小化损失函数），但是你对地图一窍不通。梯度下降就像你的导师，他不断给你指明正确的方向（调整参数），让你一步步接近那个宝藏（最小化损失函数）。', 'generated_tokens_count': 99}
